In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data

from matrix_factorization import NCF, NCF_CVIB, NCF_IPS, NCF_SNIPS, NCF_DR

from utils import gini_index, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"
# dataset_name = "yahoo"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300


In [2]:
"NCF CVIB"
ncf_cvib = NCF_CVIB(num_user, num_item)
ncf_cvib.fit(x_train, y_train, lr=0.01, 
    alpha=1e-1, gamma=1e-4, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)

test_pred = ncf_cvib.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-CVIB]" + "***"*5)
print("[NCF-CVIB] test mse:",mse_ncf)
print("[NCF-CVIB] test auc:", auc_ncf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-CVIB]" + "***"*5)

[NCF-CVIB] epoch:0, xent:8.99670535326004
[NCF-CVIB] epoch:10, xent:7.126492977142334
[NCF-CVIB] epoch:18, xent:6.419546186923981
***************[NCF-CVIB]***************
[NCF-CVIB] test mse: 0.20166660333449452
[NCF-CVIB] test auc: 0.7713471429267398
Num User: 290
Gini index: 0.3402743888402935
Global utility: 0.516551724137931
***************[NCF-CVIB]***************


In [3]:
"NCF naive"
ncf = NCF(num_user, num_item)
ncf.fit(x_train, y_train, lr=0.05, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)
test_pred = ncf.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF]" + "***"*5)
print("[NCF] test mse:", mse_func(y_test, test_pred))
print("[NCF] test auc:", auc_ncf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF]" + "***"*5)

[NCF] epoch:0, xent:9.007273197174072
[NCF] epoch:9, xent:6.504551500082016
***************[NCF]***************
[NCF] test mse: 0.20301297499383303
[NCF] test auc: 0.768840103962467
Num User: 290
Gini index: 0.3345932507711441
Global utility: 0.516551724137931
***************[NCF]***************


In [4]:
"NCF IPS"
ncf_ips = NCF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_ips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)

test_pred = ncf_ips.predict(x_test)
mse_ncfips = mse_func(y_test, test_pred)
auc_ncfips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-IPS]" + "***"*5)
print("[NCF-IPS] test mse:", mse_ncfips)
print("[NCF-IPS] test auc:", auc_ncfips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-IPS]" + "***"*5)

[NCF-IPS] epoch:0, xent:114.34569358825684
[NCF-IPS] epoch:10, xent:82.06969213485718
[NCF-IPS] epoch:20, xent:74.03709554672241
[NCF-IPS] epoch:30, xent:68.91228532791138
[NCF-IPS] epoch:32, xent:68.65648984909058
***************[NCF-IPS]***************
[NCF-IPS] test mse: 0.20081778276311885
[NCF-IPS] test auc: 0.7707961665966832
Num User: 290
Gini index: 0.34558318450336656
Global utility: 0.5193103448275862
***************[NCF-IPS]***************


In [5]:
"NCF SNIPS"
ncf_snips = NCF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_snips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)

test_pred = ncf_snips.predict(x_test)
mse_ncfsnips = mse_func(y_test, test_pred)
auc_ncfsnips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)
print("[NCF-SNIPS] test mse:", mse_ncfsnips)
print("[NCF-SNIPS] test auc:", auc_ncfsnips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)


[NCF-SNIPS] epoch:0, xent:9.024765431880951
[NCF-SNIPS] epoch:10, xent:7.861968219280243
[NCF-SNIPS] epoch:17, xent:6.318820536136627
***************[NCF-SNIPS]***************
[NCF-SNIPS] test mse: 0.19215682722874983
[NCF-SNIPS] test auc: 0.7694950891576365
Num User: 290
Gini index: 0.3303159036471287
Global utility: 0.5262068965517241
***************[NCF-SNIPS]***************


In [6]:
"NCF DR"
ncf_dr = NCF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_dr.fit(x_train, y_train, y_ips=y_ips, batch_size=512, lr=0.05, lamb=1e-3,verbose=0)
test_pred = ncf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-DR]" + "***"*5)
print("[NCF-DR] test mse:", mse_mfdr)
print("[NCF-DR] test auc:", auc_mfdr)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-DR]" + "***"*5)

D:\Counterfactual\CVIB\matrix_factorization.py:1108: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.
  xent_loss = F.binary_cross_entropy(pred, sub_y, weight=inv_prop, reduction="sum")


[NCF-DR] epoch:17, xent:31191.924560546875
***************[NCF-DR]***************
[NCF-DR] test mse: 0.21605737538658742
[NCF-DR] test auc: 0.7513524052340045
Num User: 290
Gini index: 0.3624425287356329
Global utility: 0.496551724137931
***************[NCF-DR]***************
